# Prepare Titanic Model for API / Web

Final result on Heroku:
https://titanic-model-app.herokuapp.com/

Sources:
1. https://www.kaggle.com/c/titanic/data

2. https://towardsdatascience.com/create-an-api-to-deploy-machine-learning-models-using-flask-and-heroku-67a011800c50?gi=30b632ffd17d

2. https://blog.cambridgespark.com/deploying-a-machine-learning-model-to-the-web-725688b851c7


In [14]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>
# Align table to left

# Version Control
| Version | Date | Description         
| :- |:------------- | :-
|1.0 |  2/3/2020 | added version control and released to production

# Issue log
| Date | Description | Status
| :- |:------------- | :-
| 2/3/2020 | train test split for data  | Not started
| 2/3/2020 | add cross validation | Not started

In [15]:
# last updated
from datetime import datetime
print("last updated: " + str(datetime.now()))

last updated: 2020-02-03 22:17:04.044694


## Import data

In [16]:
import pandas as pd
from sklearn.linear_model import LogisticRegression


In [17]:
# create df,
data = pd.read_csv('titanic_train.csv')  # change file path as necessary

## Data exploration, including creating sub-training set

In [18]:
data.shape

(891, 12)

In [19]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [20]:
data.describe(include='all') 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"McCarthy, Mr. Timothy J",male,NaN,NaN,NaN,1601,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [21]:
# see which is na
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [22]:
data = data.drop(['Name','Cabin','Embarked','Ticket'],axis = 1)
data.shape

(891, 8)

In [23]:
# one-hot coding 
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore',sparse = False)

nonNumData = data.select_dtypes(exclude = ['int64','float64'])
print(nonNumData.dtypes)

Sex    object
dtype: object


In [24]:
# transformed data is ok because we've remvoed na's
nonNumDataEncArray = enc.fit_transform(nonNumData) 
nonNumDataEnc = pd.DataFrame(data = nonNumDataEncArray)

In [25]:
# non Numeric data
print(list(nonNumDataEnc.columns))
featuresNonNumeric = ['Sex-' + str(x) for x in list(nonNumDataEnc.columns)]
print(enc.categories_)
print(featuresNonNumeric)
nonNumDataEnc.columns = featuresNonNumeric

# add new data back to dataset
data = pd.concat([data, nonNumDataEnc], axis=1)

[0, 1]
[array(['female', 'male'], dtype=object)]
['Sex-0', 'Sex-1']


## Feature selection

In [26]:
# features and target
target = 'Survived'
features = ['Pclass', 'Age', 'SibSp', 'Fare'] + featuresNonNumeric # X matrix, y vector

## Create and train model

In [27]:
# data is encoded with one-hot
train = data

In [28]:
# drop null values
train.dropna(inplace=True)

X = train[features]
y = train[target]# model 
model = LogisticRegression(solver='lbfgs')
model.fit(X, y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
# model coefs
print(X.columns)
print(model.coef_)
print(model.intercept_)

Index(['Pclass', 'Age', 'SibSp', 'Fare', 'Sex-0', 'Sex-1'], dtype='object')
[[-1.21242063 -0.04286476 -0.379742    0.00210823  1.27153997 -1.27192222]]
[3.94042973]


## Evaluate model

In [30]:
# score is mean accuracy
model.score(X, y)

0.8053221288515406

In [31]:
# roc curve
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y,model.predict(X) ) # roc_curve(actual, expected)
roc_auc = auc(fpr, tpr)


In [32]:
# plot roc curve
import matplotlib.pyplot as plt
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

<Figure size 640x480 with 1 Axes>

## Save model as pickle

In [33]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

## Run app.py

type "python app.py" in console while in folder

## Test app web interface
Type "flask run" in terminal, once in this directory

## Test app API


In [34]:
# local url
url = 'http://127.0.0.1:5000' 

In [35]:
# sample data
input_sample = {'Pclass': 3
      , 'Age': 2
      , 'SibSp': 1
      , 'Fare': 50}
input_sample = json.dumps(input_sample)

In [36]:
import requests, json

#send_request = requests.post(url, input_sample)
#print(send_request)

In [37]:
# check actual result
#print(send_request.json())

to stop app, press ctrl+c in console

## Then create Procfile for Heroku app

## Then create requirements.txt

Use:
Flask==1.1.1
gunicorn==19.9.0
pandas==0.25.0
requests==2.22.0
scikit-learn==0.21.2
scipy==1.3.1

More generally, can do:
pip freeze > requirements.txt

## Deploy on Heroku

## Check Heroku

In [38]:
# heroku url
heroku_url = 'https://titanic-model-app.herokuapp.com/' # change to your app name# sample data
input_sample_api = {  'Pclass': 3
             , 'Age': 2
             , 'SibSp': 1
             , 'Fare': 50}
input_sample_api = json.dumps(input_sample_api)

In [39]:
# may need to disable firewall for this
#send_request = requests.post(heroku_url, data)
#print(send_request)

In [40]:
#print(send_request.json())